# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [16]:
publications = pd.read_csv("publications.csv", sep=",", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,3/27/2024,Interpreting Time Series Transformer Models an...,AAAI'24 WORKSHOP: AI FOR TIME SERIES ANALYSIS ...,"Md. Khairul Islam, Tyler Valentine, Timothy Jo...",NaN,NaN,https://arxiv.org/html/2401.15119v1,https://github.com/UVA-MLSys/COVID-19-age-groups
1,3/23/2024,Temporal Dependencies and Spatio-Temporal Patt...,Twenty-Ninth AAAI/SIGAI Doctoral Consortium,"Md. Khairul Islam, and Judy Fox.",NaN,NaN,https://ojs.aaai.org/index.php/AAAI/article/vi...,https://github.com/UVA-MLSys/SA-Timeseries
2,7/8/2023,Interpreting County-Level COVID-19 Infections ...,IEEE International Conference on Digital Health,"Md. Khairul Islam, Yingzheng Liu, Andrej Erkel...",NaN,NaN,https://ieeexplore.ieee.org/abstract/document/...,https://github.com/UVA-MLSys/gpce-covid
3,2/13/2023,Does Differential Privacy Impact Bias in Pretr...,AAAI Workshop on Privacy-Preserving Artificial...,"Md Khairul Islam, Andrew Wang, Tianhao Wang, Y...",NaN,NaN,https://drive.google.com/file/d/1fLQIK3Z5IEOBq...,https://github.com/khairulislam/DP-on-NLP-Bias
4,5/9/2023,MVAM: Multi-variant Attacks on Memory for IoT ...,Proceedings of Cyber-Physical Systems and Inte...,"Arup Kumar Sarker, Md. Khairul Islam, Yuan Tia...",NaN,NaN,https://dl.acm.org/doi/pdf/10.1145/3576914.358...,https://github.com/arupcsedu/MVAM
5,2/1/2022,Early prediction for merged vs abandoned code ...,Information and Software Technology,"Islam K, Ahmed T, Shahriyar R, Iqbal A, Uddin G.",NaN,NaN,https://www.sciencedirect.com/science/article/...,https://github.com/khairulislam/Predict-Code-C...
6,11/27/2020,Network Anomaly Detection Using LightGBM: A Gr...,International Telecommunication Networks and A...,"Md. Khairul Islam, Prithula Hridi, M. S. Hossa...",NaN,NaN,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,https://github.com/khairulislam/Anomaly-Detect...


In [17]:
publications.fillna('', inplace=True)

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [18]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
for row, item in publications.iterrows():
    break

In [22]:
import os
from datetime import datetime

for row, item in publications.iterrows():
    # convert the format from '%m/%d/%Y' to '%Y-%m-%d'
    html_filename = datetime.strptime(item.pub_date, '%m/%d/%Y').strftime('%Y-%m-%d')
    
    md_filename = html_filename + ".md"
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + html_filename # '/' format doesn't work with yaml
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    if len(str(item.citation)) > 5:
        md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    # if len(str(item.excerpt)) > 5:
    #     md += "\n" + html_escape(item.excerpt) + "\n"

    # if len(str(item.paper_url)) > 5:
    #     md += "\n[Paper](" + item.paper_url + ")\n" 
        
    # if len(str(item.slides_url)) > 5:
    #     md += "\n[Code](" + item.slides_url + ")\n" 
    
    # if len(str(item.citation)) > 5:
    #     md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
